In [19]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [20]:
from torch import nn
import torch
import torch.nn.functional as F

In [21]:
tokenzier=AutoTokenizer.from_pretrained("bert-base-uncased")

In [22]:
model=AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [37]:
class myppl(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.tokenzier=tokenzier
        self.model=model
        self.softmax=nn.Softmax(dim=1)

    def forward(self,context,question):
        context_ids=self.tokenzier.encode(context,padding="max_length",return_tensors="pt")
        question_ids=self.tokenzier.encode(question,padding="max_length",return_tensors="pt")
        output=self.model(context_ids,question_ids)
        output=self.softmax(output[0])
        output=output.argmax(dim=1).tolist()
        ans=self.tokenzier.decode(output)
        return ans

In [38]:
ppl=myppl()
ppl("""Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a 
    question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune 
    a model on a SQuAD task, you may leverage the examples/question-answering/run_squad.py script.
    ""","What is extractive question answering?")

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, 

'[PAD]'

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

In [ ]:
tokenzier=AutoTokenizer.from_pretrained("bert-base-uncased")
model=AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# 1
from transformers import pipeline
nlp=pipeline("sentiment-analysis",model=model,tokenizer=tokenzier)

In [ ]:
nlp("I am very happy!")[0]

{'label': 'LABEL_1', 'score': 0.6212447285652161}

In [ ]:
tokenzier.batch_encode_plus(["I am very happy!","I am very sad!"],padding=True)

{'input_ids': [[101, 1045, 2572, 2200, 3407, 999, 102], [101, 1045, 2572, 2200, 6517, 999, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
model(**tokenzier.batch_encode_plus(["I am very happy!","I am very sad!"],padding=True,return_tensors="pt"))

SequenceClassifierOutput(loss=None, logits=tensor([[-0.4359,  0.0590],
        [-0.3953,  0.0361]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
class myppl(nn.Module):
    def __init__(self):
        super().__init__()
        self.tokenizer=tokenzier
        self.model=model
        self.softmax=nn.Softmax(dim=1)

    def forward(self,context):
        output=self.model(**self.tokenizer.batch_encode_plus(context,padding=True,return_tensors="pt"))
        output=self.softmax(output[0])
        output=output.argmax(dim=1).tolist()
        return output

In [ ]:
ppl=myppl()
ppl(["I am very happy!","I am very sad!"])

In [40]:
from transformers import pipeline, Conversation
converse = pipeline("conversational")

conversation_1 = Conversation("Going to the movies tonight - any suggestions?")
conversation_2 = Conversation("What's the last book you have read?")
converse([conversation_1, conversation_2])

No model was supplied, defaulted to microsoft/DialoGPT-medium and revision 8bada3b (https://huggingface.co/microsoft/DialoGPT-medium).
Using a pipeline without specifying a model name and revision in production is not recommended.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generat

[Conversation id: d9f6521c-316c-4679-9f28-afd43ba3e4ba 
 user >> Going to the movies tonight - any suggestions? 
 bot >> The Big Lebowski ,
 Conversation id: 6b1d4032-ff51-40e9-b5f3-7cf77863457e 
 user >> What's the last book you have read? 
 bot >> The Last Question ]

In [43]:
from transformers import BertGenerationEncoder, BertGenerationDecoder, BertTokenizer
from transformers import EncoderDecoderModel

In [44]:
tokenzier=BertTokenizer.from_pretrained("bert-base-uncased")
encoder=BertGenerationEncoder.from_pretrained("bert-base-uncased", bos_token_id=101, eos_token_id=102)
decoder=BertGenerationDecoder.from_pretrained("bert-base-uncased", bos_token_id=101, eos_token_id=102)
model=EncoderDecoderModel(encoder=encoder,decoder=decoder)

You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertGenerationEncoder: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'bert.pooler.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'bert.embeddings.token_type_embeddings.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertGenerationEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertGenerationEncoder from the checkpoint of a 

In [45]:
import torch

In [52]:
# model(**tokenzier.batch_encode_plus(["I am very happy!","I am very sad!"],padding=True,return_tensors="pt"))
input_ids=tokenzier("""Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a 
    question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune 
    a model on a SQuAD task, you may leverage the examples/question-answering/run_squad.py script.
    """).input_ids
labels=tokenzier("What is extractive question answering?").input_ids
output=model(input_ids=torch.tensor([input_ids]),decoder_input_ids=torch.tensor([labels]),labels=torch.tensor([labels]))

d:\Python\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


In [54]:
output.logits.shape

torch.Size([1, 9, 30522])

In [59]:
output=nn.Softmax(dim=1)(output.logits)[0]

In [60]:
output.shape

torch.Size([9, 30522])

In [61]:
output.argmax(dim=1).tolist()

[3023, 5684, 28578, 14817, 3512, 3160, 27683, 1029, 29025]

In [62]:
tokenzier.decode(_)

'coast favorrem extractive question tortricidae? pau'

In [18]:
encoder = BertGenerationEncoder.from_pretrained("bert-large-uncased", bos_token_id=101, eos_token_id=102)
decoder = BertGenerationDecoder.from_pretrained(
    "bert-large-uncased", add_cross_attention=True, is_decoder=True, bos_token_id=101, eos_token_id=102
)
bert2bert = EncoderDecoderModel(encoder=encoder, decoder=decoder)

tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")

input_ids = tokenizer("This is a long article to summarize", add_special_tokens=False, return_tensors="pt").input_ids
labels = tokenizer("This is a short summary", return_tensors="pt").input_ids

output=bert2bert(input_ids=input_ids,decoder_input_ids=labels,labels=labels)

You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertGenerationEncoder: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'bert.pooler.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'bert.embeddings.token_type_embeddings.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertGenerationEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertGenerationEncoder from the checkpoint of a

In [11]:
from torch import nn

In [32]:
nn.Softmax(dim=1)(output.logits)[0]

tensor([[0.1420, 0.1420, 0.1421,  ..., 0.1422, 0.1429, 0.1426],
        [0.1435, 0.1437, 0.1434,  ..., 0.1430, 0.1415, 0.1419],
        [0.1437, 0.1425, 0.1436,  ..., 0.1441, 0.1422, 0.1433],
        ...,
        [0.1456, 0.1440, 0.1449,  ..., 0.1460, 0.1415, 0.1429],
        [0.1427, 0.1442, 0.1428,  ..., 0.1423, 0.1469, 0.1454],
        [0.1423, 0.1428, 0.1425,  ..., 0.1420, 0.1437, 0.1425]],
       grad_fn=<SelectBackward0>)

In [33]:
_.argmax(dim=1).tolist()

[14756, 29672, 9231, 6032, 15728, 24850, 24966]

In [34]:
tokenizer.decode(_)

'##pieħ pin maintenancearuheld wept'